In [1]:
# pip install wfdb

In [2]:
import wfdb
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [3]:
#print('Current directory is: {}'.format(os.getcwd()))

In [4]:
records = []
properties = []
annot = []
AnnSymb = []
AnnSamp = []
AnnRhythm = []
Rpeak_Samp = []
Rpeak_Symb = []


for f in glob.glob('C:/Users/masud/Desktop/Github_Bachelor_Project/wfdb-python/data/*.dat'): ##### change the path (afib) to your own directory. The rest should 
                                                                                             ##### remin the same
    sig, fields = wfdb.rdsamp(f[:-4])  #### In this function, you have the opportunity to pass 
                                       #### "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'qrs')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records.append(sig)
    properties.append(fields)
    annot.append(ann)
    AnnSymb.append(Symb)
    AnnSamp.append(Samp)
    Rpeak_Symb.append(QRS_Symb)
    Rpeak_Samp.append(QRS_Samp)
    AnnRhythm.append(Rhythm)
    
    

AnnSymb = pd.Series(AnnSymb).values
AnnSamp = pd.Series(AnnSamp).values

In [69]:
#This is only for last record, calculated in above loop
print(ann.aux_note)
print(QRS.sample)

['(N', '(AFIB', '(N', '(AFIB']
[    193     390     586 ... 8999879 8999948 9000001]


In [61]:
single_record_rri = []
rri = []
single_record_mean = []
rri_mean = []
single_record_std = []
rri_std = []
single_record_RMSSD = []
RMSSD = []
single_record_nRMSSD = []
nRMSSD = []


i = 0
for item in Rpeak_Samp:
    single_record_rri = np.diff(Rpeak_Samp[i])
    rri.append(single_record_rri)
    single_record_mean = np.mean(single_record_rri)
    rri_mean.append(single_record_mean)
    single_record_std = np.std(Rpeak_Samp[i])
    rri_std.append(single_record_std)
    
    # to estimate HRV in a specified intervals of an ECG
    #Root mean square of successive differences = RMSSD

    #range -1, gives no error out of bound, because of index i+1 inside the loop. (range = 59550)
    for k in range(len(single_record_rri)-1):
        squared_summed_differencies += (single_record_rri[k+1] - single_record_rri[k])**2

    squared_summed_differencies_multiplied = 1/(len(single_record_rri)-1) * squared_summed_differencies
    single_record_RMSSD = math.sqrt(squared_summed_differencies_multiplied)
    RMSSD.append(single_record_RMSSD)
    single_record_nRMSSD = (single_record_RMSSD / single_record_mean)
    nRMSSD.append(single_record_nRMSSD)
    
    squared_summed_differencies = 0
    i += 1

In [70]:
#Only result for last record
print(f"R-R intervals: {single_record_rri}")
print(f"Mean: {single_record_mean}")
print(f"Standard deviation: {single_record_std}")
print(f"RMSSD: {single_record_RMSSD}")
print(f"Normalized RMSSD: {single_record_nRMSSD}")

R-R intervals: [197 196 196 ...  68  69  53]
Mean: 151.12773924871118
Standard deviation: 2450606.0325444057
RMSSD: 42.482330598711705
Normalized RMSSD: 0.28110213789937305


In [ ]:
#A sample of 250 elements, gives 1 clear signal (because 250hz frequency)
#x = np.arange(0,250)
#plt.plot(x,rri[750:1000])